# sentence transformers 임베딩 테스트용 파일

## 임베딩 모델 설정 및 임베딩 대상 파일 import

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model= SentenceTransformer('paraphrase-MpNet-base-v2')

df = pd.read_csv('./0624_final_df_with_summaries_1~757.csv')
df['임베딩결과값'] = None
total_count = len(df)

C:\Users\thdth\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## 임베딩 실행

In [ ]:
# 'review_summarized' 열의 텍스트를 임베딩하여 새로운 열 '임베딩결과값'에 저장
for index, row in df.iterrows():
    text = row['review_summarized']
    embedding = model.encode(text)
    
    # 임베딩 결과를 '임베딩결과값' 열에 할당
    df.at[index, '임베딩결과값'] = embedding.tolist()  # 리스트로 변환하여 저장
    
    # 진행 상황 출력
    print(f"Embedding progress: {index + 1}/{len(df)}")
    
    # 매 반복마다 중간 결과 저장
    if (index + 1) % 100 == 0:
        df.to_csv('./0624_final_df_with_summaries_1~757_st임베딩중간결과.csv', encoding='utf-8-sig', index=False)

# 최종 결과 저장
df.to_csv('./0624_final_df_with_summaries_1~757_st임베딩완료.csv', encoding='utf-8-sig', index=False)

## 임베딩 파일 불러오기

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import openai
import ast

df = pd.read_csv('./0624_final_df_with_summaries_1~757_st임베딩완료.csv')
# 임베딩 결과값을 리스트 형식으로 변환
# 임베딩결과값 열을 리스트로 변환하여 코사인 유사도를 계산할 수 있도록 준비하기 위함
df['임베딩결과값'] = df['임베딩결과값'].apply(ast.literal_eval)

## 코사인 유사도를 활용한 추천시스템 테스트

In [4]:
# 임베딩 함수 정의
def get_text_embedding(text):
    return model.encode(text)

# 추천 시스템 함수 정의
def recommend_documents(prompt, df, top_n=3):
    # 입력 텍스트 임베딩
    input_embedding = get_text_embedding(prompt)
    
    # 코사인 유사도 계산
    embeddings = list(df['임베딩결과값'])
    similarities = cosine_similarity([input_embedding], embeddings)[0]
    
    # 유사도 높은 순으로 정렬하여 상위 n개 추천
    top_indices = similarities.argsort()[-top_n:][::-1]  # 유사도가 높은 순으로 인덱스 정렬
    
    # 상위 n개 추천 음식점 이름 리스트와 유사도
    recommended_restaurants = []
    for idx in top_indices:
        restaurant_info = {
            'restaurant_name': df.iloc[idx]['restaurant_name'],
            'similarity': similarities[idx]
        }
        recommended_restaurants.append(restaurant_info)
    
    return recommended_restaurants

# 사용자 입력 받기
user_input = input("추천을 받고 싶은 음식점을 입력하세요: ")

# 추천 실행
recommendation = recommend_documents(user_input, df)
for i, rec in enumerate(recommendation, 1):
    print(f"추천 음식점 {i}: {rec['restaurant_name']} (유사도: {rec['similarity']:.4f})")

추천을 받고 싶은 음식점을 입력하세요: 고기 구워주는 식당 알려줘
추천 음식점 1: 옥정 (유사도: 0.6631)
추천 음식점 2: 샐러드바스켓 신촌점 (유사도: 0.6578)
추천 음식점 3: 도회푸드 건강한쌈 홍대점 (유사도: 0.6578)
